In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import json

def get_html_from_url(url):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.binary_location = (
        '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    )

    # Initialize a Chrome webdriver
    driver = webdriver.Chrome(
        executable_path='/Users/lizhicq/Applications/Drivers/chromedriver', 
        options=chrome_options
    )
    # Load the page
    driver.get(url)

    # Get the source HTML of the page
    html = driver.page_source

    # Close the browser
    driver.quit()

    return html

def parse_html_to_json(html):
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    exhibitor_dict = {}

    # Find all "li-inner" divs
    exhibitor_divs = soup.find_all('div', class_='li-inner')

    for exhibitor_div in exhibitor_divs:
        # Extract the exhibitor name and booth number from the "chapter" spans
        chapter_spans = exhibitor_div.find_all('span', class_='chapter')
        toc_page_spans = exhibitor_div.find_all('span', class_='toc-page')

        if chapter_spans and toc_page_spans:
            # Assume the first "chapter" span contains the exhibitor name
            name = chapter_spans[0].get_text(strip=True)

            # Assume the first "toc-page" span contains the booth number
            booth = toc_page_spans[0].get_text(strip=True) 

            # Add the name and booth number to the dictionary
            if name and booth:
                exhibitor_dict[name] = booth

    # Convert the dictionary to a JSON string
    exhibitor_json = json.dumps(exhibitor_dict, ensure_ascii=False)
    
    return exhibitor_json


if __name__ == "__main__":
    # Get HTML from the URL
    url = 'https://www.chinajoy.net/cgzn/wycg/zslb/'
    html = get_html_from_url(url)

    # Parse HTML to JSON
    exhibitor_json = parse_html_to_json(html)

    # Print the JSON string
    print(exhibitor_json)


{"CASCADE STUDIOS": "现场特别展示", "南昌众瑟芬芸游戏科技有限公司": "W5-IG097", "心智模型工作室": "W5-IG096", "厦门蜜糖时光信息科技有限公司": "W5-IG095", "厦门蜜糖时光": "W5-IG095", "济南天桥区喵斯塔卜软件科技工作室": "W5-IG094", "杭州头号互娱科技有限公司": "W5-IG093", "北京漫游谷信息技术有限公司": "W5-IG092", "Curve Games": "W5-IG091", "南京早上好软件有限公司": "W5-IG090", "腾浊流": "W5-IG089", "四十四工作室": "W5-IG088", "迷木工作室": "W5-IG087", "成都乙钛科技有限公司": "W5-IG086", "红河游点好玩科技服务有限公司": "W5-IG085", "安米翻译": "W5-IG084", "声魂（深圳）文化科技有限公司": "W5-IG083", "上海唯偲科技信息有限公司": "W5-IG082", "谦游坊": "W5-IG080/081", "重庆乐构思科技有限责任公司": "W5-IG079"}


In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import json, time
import pandas as pd

def get_html(url):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    
    # Initialize a Chrome webdriver
    driver = webdriver.Chrome(executable_path='/Users/lizhicq/Applications/Drivers/chromedriver', options=chrome_options)
    
    # Load the page
    driver.get(url)

    # Initialize a list to hold exhibitor data from all pages
    all_exhibitors = []
    
    count = 1
    previous_dict = None
    repeat_count = 0
    while True:
        # Get the source HTML of the page
        html = driver.page_source
        
        # Parse the HTML and get exhibitor data
        exhibitor_dict = parse_html(html)

        # If the exhibitor data didn't change, we might be on the last page
        if exhibitor_dict == previous_dict:
            repeat_count += 1
            if repeat_count == 3:
                break
        else:
            repeat_count = 0
            previous_dict = exhibitor_dict
            print(f'Page {count}: {json.dumps(exhibitor_dict, ensure_ascii=False)}')
            count += 1
            all_exhibitors.append(exhibitor_dict)  # Add the exhibitor data to the list
        
        # Try to go to the next page
        try:
            driver.execute_script('nextPage()')
            time.sleep(1)  # Wait for 5 seconds for the page to load
        except:
            break  # Exit the loop if we can't go to the next page
    
    # Close the browser
    driver.quit()
  
    # Merge all exhibitor data into one dictionary
    merged_exhibitors = {k: v for exhibitor_dict in all_exhibitors for k, v in exhibitor_dict.items()}

    # Convert the dictionary to a pandas DataFrame and print it
    df = pd.DataFrame(list(merged_exhibitors.items()), columns=['Exhibitor', 'Booth'])
    
    # Use tabulate to create a formatted table
    return df
   


def parse_html(html):
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    exhibitor_dict = {}

    # Find all "li" tags
    exhibitor_lis = soup.find_all('li')

    for exhibitor_li in exhibitor_lis:
        # Extract the exhibitor name and booth number from the "chapter" and "toc-page" spans
        chapter_span = exhibitor_li.find('span', class_='chapter')
        toc_page_span = exhibitor_li.find('span', class_='toc-page')

        if chapter_span and toc_page_span:
            # Extract the text from the spans
            name = chapter_span.get_text(strip=True)
            booth = toc_page_span.get_text(strip=True)
            
            # Add the name and booth number to the dictionary
            exhibitor_dict[name] = booth
    
    return exhibitor_dict


# Target URL
url = 'https://www.chinajoy.net/cgzn/wycg/zslb/'

# Get HTML and parse it
df = get_html(url)

Page 1: {"CASCADE STUDIOS": "现场特别展示", "南昌众瑟芬芸游戏科技有限公司": "W5-IG097", "心智模型工作室": "W5-IG096", "厦门蜜糖时光信息科技有限公司": "W5-IG095", "厦门蜜糖时光": "W5-IG095", "济南天桥区喵斯塔卜软件科技工作室": "W5-IG094", "杭州头号互娱科技有限公司": "W5-IG093", "北京漫游谷信息技术有限公司": "W5-IG092", "Curve Games": "W5-IG091", "南京早上好软件有限公司": "W5-IG090", "腾浊流": "W5-IG089", "四十四工作室": "W5-IG088", "迷木工作室": "W5-IG087", "成都乙钛科技有限公司": "W5-IG086", "红河游点好玩科技服务有限公司": "W5-IG085", "安米翻译": "W5-IG084", "声魂（深圳）文化科技有限公司": "W5-IG083", "上海唯偲科技信息有限公司": "W5-IG082", "谦游坊": "W5-IG080/081", "重庆乐构思科技有限责任公司": "W5-IG079"}
Page 2: {"重庆帕斯亚科技有限公司": "W5-IG077/078", "一罐盐（北京）科技有限公司": "W5-IG074/075", "噬零工作室": "W5-IG073", "铃兰之径工作室": "W5-IG072", "赤原度": "W5-IG071", "无锡九天云游网络科技有限公司": "W5-IG070/076", "归零者工作室": "W5-IG069", "663 Games": "W5-IG068", "上海锌仪信息科技有限公司": "W5-IG067", "北京黎羽科技有限公司": "W5-IG062/063/064", "上海仙仙兔网络科技有限公司": "W5-IG061", "武汉羽风文化": "W5-IG060", "重庆克鲁格网络技术有限公司": "W5-IG059", "南通星椽网络科技有限公司": "W5-IG057", "Luna Orion工作室": "W5-IG056", "亚斯特尔动力": "W5-IG055", "无明工作室": "W5-IG053/054/065/0

In [37]:
from IPython.display import HTML
# Use the Styler class to render the DataFrame as an HTML table
html_table = df.style.set_table_attributes('style="font-size: 14px; text-align: left; border: 1px solid black"').render()

# Display the HTML table
HTML(html_table)


/var/folders/1k/xjqy__5j3gg35387ft__8k540000gn/T/ipykernel_5428/1333672276.py:3: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html_table = df.style.set_table_attributes('style="font-size: 14px; text-align: left; border: 1px solid black"').render()


,Exhibitor,Booth
0,CASCADE STUDIOS,现场特别展示
1,南昌众瑟芬芸游戏科技有限公司,W5-IG097
2,心智模型工作室,W5-IG096
3,厦门蜜糖时光信息科技有限公司,W5-IG095
4,厦门蜜糖时光,W5-IG095
5,济南天桥区喵斯塔卜软件科技工作室,W5-IG094
6,杭州头号互娱科技有限公司,W5-IG093
7,北京漫游谷信息技术有限公司,W5-IG092
8,Curve Games,W5-IG091
9,南京早上好软件有限公司,W5-IG090
